In [1]:
import sys
import os
sys.path.append("..")
from knool.remote_sensing.satellite import tles_for_asat
from knool.geodata_processor import geo_info
import pandas as pd
from skyfield.api import load
from datetime import datetime as dt
from datetime import timedelta
from skyfield.api import utc
import numpy as np
from osgeo import gdal, osr, ogr

In [2]:
##READ tle file. https://qiita.com/grinpeaceman/items/22f06714a98dbfacce88

tles = load.tle_file("../knool/remote_sensing/data/two_line_elements/sat40697.txt") # TLE読み込み
satellite=tles[0]
sgp4_model = satellite.model

print("衛星名: ", satellite.name)
print("元期: ", satellite.epoch.utc_jpl())
print("衛星番号: ", sgp4_model.satnum)
print("エポック年: ", sgp4_model.epochyr)
print("エポック日: ", sgp4_model.epochdays)
print("エポックのユリウス日: ", sgp4_model.jdsatepoch)
print("ndot: ", sgp4_model.ndot)
print("nddot: ", sgp4_model.nddot)
print("弾道抗力係数 B*: ", sgp4_model.bstar)
print("軌道傾斜角[rad]: ", sgp4_model.inclo)
print("昇交点赤経[rad]: ", sgp4_model.nodeo)
print("離心率: ", sgp4_model.ecco)
print("近地点引数[rad]: ", sgp4_model.argpo)
print("平均近点角[rad]: ", sgp4_model.mo)
print("平均運動[rad/min]: ", sgp4_model.no_kozai)

衛星名:  None
元期:  A.D. 2015-Jun-23 03:50:23.3849 UTC
衛星番号:  40697
エポック年:  15
エポック日:  174.15999288
エポックのユリウス日:  2457196.5
ndot:  -1.333237623051224e-12
nddot:  0.0
弾道抗力係数 B*:  0.0
軌道傾斜角[rad]:  1.7206276070908537
昇交点赤経[rad]:  4.337586504835915
離心率:  7.98e-05
近地点引数[rad]:  4.953066393576204
平均近点角[rad]:  2.5043519997356394
平均運動[rad/min]:  0.06244205289471824


In [3]:
#get lat,lon,elevation from a tle
ts = load.timescale()  # skyfiled上で時刻を扱うモジュール
tle=tles[0]
date1 = dt(2018, 2, 1, 12, 15, 30, 2000, tzinfo=utc)
date1_sf=ts.from_datetime(date1)

geocentric=tle.at(date1_sf) #get xyz
subpoint = geocentric.subpoint()
lat = subpoint.latitude.degrees
lon = subpoint.longitude.degrees
ele = subpoint.elevation.m
print(geocentric.velocity)
print(geocentric.position)
print(geocentric.xyz)
print(lat,lon,ele)

[-0.00141461  0.00399833  0.00070632] au/day
[-9.51255273e-06  4.88624241e-06 -4.67440961e-05] au
[-9.51255273e-06  4.88624241e-06 -4.67440961e-05] au
-77.27650275778588 -162.68188667619506 815689.5183005581


In [4]:
#convert lla/ecef from ecef/lla using our package.
print(lat,lon,ele)
x,y,z=geo_info.convert_lla_to_ecef(lat,lon,ele)
print(x,y,z) # meter
lat,lon,ele=geo_info.convert_ecef_to_lla(x,y,z)
print(lat,lon,ele)

-77.27650275778588 -162.68188667619506 815689.5183005581
-1516878.3505331383 -472981.10875354544 -6995304.199162432
-77.2765027607371 -162.68188667619506 815689.519945832


In [5]:
#create a kml file of a orbit line
infile=r"../knool/remote_sensing/data/two_line_elements/sat27424.txt"
test=tles_for_asat(infile)
test.set_period("2011-06-30 10:00:00","2011-07-02 11:40:00")
test.calc_positions_between(5"2011-07-01 10:00:00","2011-07-01 11:40:00",1)
test.export("../../test_data/output/test.kml")

c:\Users\knaka\Documents\program\kn-general_analysis_tools\notebook\..\knool\remote_sensing\satellite.py:40: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  row1=df.loc[df.index==df.index.unique()[df.index.unique().get_loc(pd.to_datetime(date), method='nearest')]]


In [6]:
#create a kml file of a orbit line
test.calc_positions_faster_between("2011-07-01 10:00:00","2011-07-01 11:40:00",1)
test.export("../../test_data/output/test2.kml")

c:\Users\knaka\Documents\program\kn-general_analysis_tools\notebook\..\knool\remote_sensing\satellite.py:65: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  row1=df.loc[df.index==df.index.unique()[df.index.unique().get_loc(pd.to_datetime(mdt), method='nearest')]]


In [7]:
#Creates a kml file of a polygon
test.calc_buff_area_between("2011-07-01 10:00:00","2011-07-01 11:00:00",1,1000000,ori="right")
test.export("../../test_data/output/test3.kml")

c:\Users\knaka\Documents\program\kn-general_analysis_tools\notebook\..\knool\remote_sensing\satellite.py:104: FutureWarning: Passing method to DatetimeIndex.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  row1=df.loc[df.index==df.index.unique()[df.index.unique().get_loc(pd.to_datetime(mdt), method='nearest')]]


In [ ]:
from datetime import datetime, timedelta, timezone

from skyfield.api import load, Topos


satellite = load.tle_file("./aqua.txt")[0]  # TLE読み込み

ts = load.timescale()  # タイムスケールを生成

# 観測地点(東京タワー)の位置情報を設定
tokyo_tower_pos = Topos(
    latitude_degrees=35.66, longitude_degrees=139.75, elevation_m=333
)

start = datetime.now(timezone.utc)
end = start + timedelta(days=1)

# 24時間以内に観測点(東京タワー)から衛星が見える時間を計算(仰角5度以上)
# パス開始時刻、最大仰角時刻、パス終了時刻の情報が取得できる
t, events = satellite.find_events(
    tokyo_tower_pos, ts.utc(start), ts.utc(end), altitude_degrees=5.0
)

difference = satellite - tokyo_tower_pos
for ti, event in zip(t, events):
    name = ("パス開始: ", "最大仰角: ", "パス終了: ")[event]
    print(name, ti.utc_strftime("[UTC] %Y %b %d %H:%M:%S"))
    if event == 1:
        topocentric = difference.at(ti)
        alt, _, _ = topocentric.altaz()  # 観測点から観た衛星の仰角を取得
        print("最大仰角[degree]: ", alt.degrees)
    if event == 2:
        print()